In [ ]:
%reload_ext autoreload
%autoreload 2
    
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import seaborn as sns
from scipy.stats import spearmanr

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.plotting import plotting_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, rf, get_session_list

In [ ]:
VERSION = 3
SAVE_ROOT = (
    Path(
        f"/Volumes/lab-znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
    )
    / "fig2"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)


In [ ]:
fname = "/Volumes/proj-znamenp-3dvision/raw/hey2_3d-vision_foodres_20220101/PZAH8.2h/S20230116/R210322_SpheresPermTubeReward/NewParams.csv"
params = pd.read_csv(fname)

In [ ]:
# Load data
# Load data for example reconstructed stimulus frame
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)

# Load data for example RF
session_name_rf = "PZAH8.2h_S20230116"
vs_df_rf, trials_df_rf = spheres.sync_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
frames_rf, imaging_df_rf = spheres.regenerate_frames_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
    resolution=1,
)
neurons_ds_rf = pipeline_utils.create_neurons_ds(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    project=project,
    conflicts="skip",
)
neurons_df_rf = pd.read_pickle(neurons_ds_rf.path_full)


In [ ]:
# # Load data for significant RF %
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
all_sig, all_sig_ipsi, neurons_df_all = rf.load_sig_rf(
    flexilims_session=flexilims_session,
    session_list=get_session_list.get_all_sessions(
        project=project,
        mouse_list=[
            "PZAH6.4b",
            "PZAG3.4f",
            "PZAH8.2h",
            "PZAH8.2f",
            "PZAH8.2i",
            "PZAH10.2d",
            "PZAH10.2f",
        ],
        closedloop_only=False,
        openloop_only=False,
    ),
    n_std=6,
)


## 3D RF


In [ ]:
# for every row of trials_df_rf, average frames_rf between imaging_stim_start and imaging_stim_stop
depths = []
mean_frames = []
for i, row in trials_df_rf.iterrows():
    depths.append(row.depth)
    mean_frames.append(
        np.mean(frames_rf[row.imaging_stim_start : row.imaging_stim_stop, :, :])
    )


In [ ]:
plt.plot(np.log(np.array(depths)), np.array(mean_frames), ".")
# plot the mean for each depth
depths = np.array(depths)
mean_frames = np.array(mean_frames)
depths_unique = np.unique(depths)
mean_frames_unique = np.zeros(len(depths_unique))
for i, depth in enumerate(depths_unique):
    mean_frames_unique[i] = np.mean(mean_frames[depths == depth])
plt.plot(np.log(depths_unique), mean_frames_unique, "o")


In [ ]:
plt.rcParams["font.family"] = "Arial"
fontsize_dict = {"title": 7, "label": 7, "tick": 6, "legend": 5}
fig = plt.figure(figsize=(18 / 2.54, 18 / 2.54))
# Fig.2A Example reconstructed stimuli frame
trial_idx = 4
example_trial = trials_df_rf.iloc[trial_idx]
depths = np.sort(trials_df_rf.depth.unique())
idepth = np.nonzero(depths == example_trial.depth)[0][0]
iframe = (trials_df_rf.iloc[trial_idx].imaging_stim_start + 10,)
rf.plot_stimulus_frame(
    frame=frames_rf[iframe, :, frames_rf.shape[2] // 2 :].squeeze(),
    idepth=idepth,
    depths=depths,
    position=[0, 0.74, 0.3, 0.4],
    fontsize_dict=fontsize_dict,
    plot_prop=0.9,
)

# Fig.2B Linear-nonlinear receptive field model schematic. (maybe not necessary)

# Fig.2C Example receptive field of 2 V1 neurons
# PZAH8.2h_S20230116: 271, 299, 319, 321, 402, 344, 356, 306, 110, 117, 86, 88, 93, 110, 117, 120, 141, 223, 250, 350, 370, 412, 434, 437
SELECT_ROIS = [591, 402, 93, 434, 88]  # change 271 to 591
for iroi, roi in enumerate(SELECT_ROIS):
    fig.add_axes([0.29 + 0.15 * iroi, 0.9, 0.05, 0.05])
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_rf,
        trials_df=trials_df_rf,
        roi=roi,
        rs_thr=None,
        plot_fit=False,
        linewidth=2,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
    )
    plt.ylabel("")
    plt.xlabel("")
    plt.gca().set_xticklabels([])
    plt.gca().tick_params(length=1.5)
    if iroi == len(SELECT_ROIS) // 2:
        xlabel = "Azimuth (degrees)"
    else:
        xlabel = ""
    if iroi == 0:
        ylabel = "Elevation (degrees)"
    else:
        ylabel = ""
    rf.plot_rf(
        neurons_df=neurons_df_rf,
        roi=roi,
        ndepths=8,
        frame_shape=(16, 24),
        position=[0.08 + 0.15 * iroi, 0.83, 0.5, 0.5],
        plot_prop=0.9,
        xlabel=xlabel,
        ylabel=ylabel,
        fontsize_dict=fontsize_dict,
    )

# Fig.2D Proportion of neurons with significant receptive fields across imaging sessions
fig.add_axes([0.1, 0.15, 0.17, 0.14])
rf.plot_sig_rf_perc(
    all_sig=all_sig,
    all_sig_ipsi=all_sig_ipsi,
    plot_type="hist",
    bar_color="k",
    bins=np.arange(0, 1, 0.1),
    fontsize_dict=fontsize_dict,
)

# Fig.2F-G Preferred azimuth & elevation follow V1 retinotopic gradients
# example session, spatial distribution of preferred azimuth/elevation/depth
# visualize azimuth & elevation on the 2p cell mask (or as a function of position)--> color-code - two separate plots --> include position in neurons_df
# Load suite2p ops files
suite2p_ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=session_name_rf,
    dataset_type="suite2p_rois",
    filter_datasets={"anatomical_only": 3},
    allow_multiple=False,
    return_dataseries=False,
)
iscell = np.load(suite2p_ds.path_full / "plane0" / "iscell.npy", allow_pickle=True)[
    :, 0
]
neurons_df_rf["iscell"] = iscell
stat = np.load(suite2p_ds.path_full / "plane0" / "stat.npy", allow_pickle=True)
ops = np.load(suite2p_ds.path_full / "plane0" / "ops.npy", allow_pickle=True).item()

fig.add_axes([0.3, 0.075, 0.25, 0.25])
depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    param="preferred_depth",
    cmap=cm.cool.reversed(),
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
)
plt.title("Preferred depth", fontsize=fontsize_dict["title"])

fig.add_axes([0.55, 0.075, 0.25, 0.25])
depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    param="preferred_azimuth",
    cmap=cm.YlOrRd,
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
)
plt.title("Preferred azimuth", fontsize=fontsize_dict["title"])

fig.add_axes([0.8, 0.075, 0.25, 0.25])
depth_selectivity.plot_example_fov(
    neurons_df=neurons_df_rf,
    ops=ops,
    stat=stat,
    ndepths=len(depths),
    param="preferred_elevation",
    cmap=cm.YlOrRd,
    background_color=np.array([0, 0, 0]),
    fontsize_dict=fontsize_dict,
)
plt.title("Preferred elevation", fontsize=fontsize_dict["title"])
plt.savefig(SAVE_ROOT / "fig2.pdf", transparent=True, bbox_inches="tight")
plt.savefig(SAVE_ROOT / "fig2.png", transparent=True, bbox_inches="tight", dpi=300)


## Preferred depth map


In [ ]:
sig_neurons = (
    (neurons_df_all["rf_sig"] == True)
    & (neurons_df_all["iscell"] == True)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
)
neurons_df_all["rf_azi_jittered"] = neurons_df_all["rf_azi"] + np.random.uniform(
    -2, 2, len(neurons_df_all)
)
neurons_df_all["rf_ele_jittered"] = neurons_df_all["rf_ele"] + np.random.uniform(
    -2, 2, len(neurons_df_all)
)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.scatter(
    neurons_df_all[sig_neurons]["rf_azi_jittered"],
    neurons_df_all[sig_neurons]["preferred_depth_closedloop"] * 100,
    s=1,
    alpha=0.25,
    c="k",
)
# bin azimuth in 5 degree steps and plot the median with confidence intervals
azimuth_bins = np.arange(0, 125, 5)
azimuth_median = np.zeros(len(azimuth_bins) - 1)
azimuth_ci = np.zeros((2, len(azimuth_bins) - 1))
for i, (az1, az2) in enumerate(zip(azimuth_bins[:-1], azimuth_bins[1:])):
    idx = (neurons_df_all[sig_neurons]["rf_azi"] >= az1) & (
        neurons_df_all[sig_neurons]["rf_azi"] < az2
    )
    azimuth_median[i] = (
        np.median(neurons_df_all[sig_neurons][idx]["preferred_depth_closedloop"]) * 100
    )
    azimuth_ci[0, i], azimuth_ci[1, i] = common_utils.get_bootstrap_ci(
        neurons_df_all[sig_neurons][idx]["preferred_depth_closedloop"].values[
            np.newaxis, :
        ]
        * 100,
        func=np.median,
    )
plt.plot(azimuth_bins[:-1] + 2.5, azimuth_median, c="k", lw=2)
plt.fill_between(
    azimuth_bins[:-1] + 2.5,
    azimuth_ci[0, :],
    azimuth_ci[1, :],
    color="k",
    alpha=0.3,
    lw=0,
)
plt.yscale("log")
plt.gca().set_aspect("auto")
plotting_utils.despine()
r, p = spearmanr(
    neurons_df_all[sig_neurons]["preferred_depth_closedloop"],
    neurons_df_all[sig_neurons]["rf_azi"],
)
plt.ylabel("Preferred depth (cm)")
plt.xlabel("RF azimuth " + "(\N{DEGREE SIGN})")
plt.title(f"R = {r:.2f}, p = {p:.3e}")

plt.subplot(1, 2, 2)
plt.scatter(
    neurons_df_all[sig_neurons]["rf_ele_jittered"],
    neurons_df_all[sig_neurons]["preferred_depth_closedloop"] * 100,
    s=1,
    alpha=0.5,
    c="k",
)
elevation_bins = np.arange(0, 125, 5)
plt.yscale("log")
plt.gca().set_aspect("auto")
plotting_utils.despine()
r, p = spearmanr(
    neurons_df_all[sig_neurons]["preferred_depth_closedloop"],
    neurons_df_all[sig_neurons]["rf_ele"],
)
plt.ylabel("Preferred depth (cm)")
plt.xlabel("RF elevation " + "(\N{DEGREE SIGN})")
plt.title(f"R = {r:.2f}, p = {p:.3e}")
plt.savefig(SAVE_ROOT / "rf_depth_correlation.pdf", bbox_inches="tight")
plt.savefig(
    SAVE_ROOT / "rf_depth_correlation.png",
    bbox_inches="tight",
    dpi=300,
    transparent=True,
)


In [ ]:
azimuth_median

In [ ]:
# relationship to preferred RS / OF


In [ ]:
# any bias between animals / between sessions?
# hiearchical bootstrap (resample animals, session, mice) --> distribution of correlation coefficients


In [ ]:
sig_neurons = (
    (neurons_df_all["rf_sig"] == True)
    & (neurons_df_all["iscell"] == True)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
)
neurons_df_all[neurons_df_all.rf_azi <= 30].groupby("session").count().sort_values(
    "roi", ascending=False
)[:20]


In [ ]:
sns.scatterplot(
    neurons_df_all[sig_neurons],
    x="rf_azi_jittered",
    y="rf_ele_jittered",
    hue=np.log(neurons_df_all[sig_neurons]["preferred_depth_closedloop"] * 100),
    # hue_norm = (np.log(6), np.log(600)),
    palette="cool_r",
    s=5,
    alpha=0.3,
    ax=plt.figure().gca(),
    linewidth=0,
    legend=False,
)


In [ ]:
neurons_df_all["preferred_depth_corrected"] = neurons_df_all[
    "preferred_depth_closedloop"
] / np.sin(np.deg2rad(neurons_df_all["rf_azi"]))

azis = np.sort(neurons_df_all["rf_azi"].unique())
eles = np.sort(neurons_df_all["rf_ele"].unique())
depth_matrix = np.zeros((len(azis), len(eles)))
depth_matrix_corrected = np.zeros((len(azis), len(eles)))
for iazi in range(len(azis)):
    for iele in range(len(eles)):
        depth_matrix[iazi, iele] = (
            neurons_df_all[sig_neurons][
                (neurons_df_all["rf_azi"] == azis[iazi])
                & (neurons_df_all["rf_ele"] == eles[iele])
            ]["preferred_depth_closedloop"].mean()
            * 100
        )
        depth_matrix_corrected[iazi, iele] = (
            neurons_df_all[sig_neurons][
                (neurons_df_all["rf_azi"] == azis[iazi])
                & (neurons_df_all["rf_ele"] == eles[iele])
            ]["preferred_depth_corrected"].mean()
            * 100
        )
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(
    np.log(depth_matrix).T,
    cmap="cool_r",
    origin="lower",
    aspect="equal",
    extent=(azis[0], azis[-1], eles[0], eles[-1]),
    vmin=np.log(6),
    vmax=np.log(600),
)
plt.title("Preferred depth (cm)")
cbar = plt.colorbar()
cbar.set_ticks(np.linspace(np.log(6), np.log(600), 5))
cbar.set_ticklabels(np.linspace(6, 600, 5).astype("int"))

plt.subplot(122)
plt.imshow(
    np.log(depth_matrix_corrected).T,
    cmap="cool_r",
    origin="lower",
    aspect="equal",
    extent=(azis[0], azis[-1], eles[0], eles[-1]),
    vmin=np.log(6),
    vmax=np.log(600),
)
plt.title("Corrected preferred depth (cm)")
cbar = plt.colorbar()
cbar.set_ticks(np.linspace(np.log(6), np.log(600), 5))
cbar.set_ticklabels(np.linspace(6, 600, 5).astype("int"))

# bias of distribution of ON/OFF RGCs in the retina


In [ ]:
# MINOR:
# - add location on overview & scalebars (can highlight the neurons above)
# - DONE: change the near cell example to 591
# - DONE: change colormap for azimuth/elevation to plasma, viridis etc summer autumn, BuYl

# CHECK:
# - DONE: double check if significant RF cells include optic flow near cells

# MAJOR:
# - summary of all sessions: compute the distance between n nearest neighbors vs n random neighbors. sklearn.neighbors.KDTree

# Other:
# - R2 distribution, add non-linear fit (no need to do it yet);


## SUPP


In [ ]:
# Fig.2E Distribution of azimuth/elevation of depth-selective neurons in 2 example imaging sessions
# Load data for plotting center of RFs
sessions_dist_exp = ["PZAH6.4b_S20220419", "PZAH6.4b_S20220426"]
results_dist_exp = rf.get_rf_results(project, sessions_dist_exp)

fig = plt.figure()
rf.plot_rf_centers(
    fig=fig,
    results=results_dist_exp,
    colors=["r", "b"],
    ndepths=5,
    frame_shape=(16, 24),
    n_stds=5,
    plot_x=0.3,
    plot_y=0,
    plot_width=0.3,
    plot_height=0.3,
    fontsize_dict={"title": 15, "label": 10, "tick": 5},
)


In [ ]:
plt.figure()
plt.hist(azi, bins=20)
plt.gca().set_aspect(0.6)
plotting_utils.despine()
plt.xlabel("Preferred azimuth (°)")

plt.figure()
plt.hist(ele, bins=20)
plt.gca().set_aspect(0.3)
plotting_utils.despine()
plt.xlabel("Preferred elevation (°)")
